In [ ]:
# Вспомогательный метод для запуска Tensorboard в Colab

# Fix: https://stackoverflow.com/questions/60730544/tensorboard-colab-tensorflow-api-v1-io-gfile-has-no-attribute-get-filesystem
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import os
import shutil

# Запуск Tensorboard в Colab
def reinit_tensorboard(clear_log = True):
  # Лог-файлы читаются из этого каталога: 
  logs_base_dir = "runs"
  if clear_log:
    # Очистка логов
    #!rm -rfv {logs_base_dir}/*
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Магия Colab
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

In [ ]:
reinit_tensorboard()

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

# Задание 1.1 

Используя код из лекции, посмотрите, как будут вести функции активации nn.Tanh и nn.ReLU в случае инициализации весов по-умолчанию использууемой PyTorch.
Конкретно обратите внимание на распределение градиентов слоев и активаций


Будет ли обучаться нейросеть из лекции с 5 слоями, если в ней заменить активации на nn.ReLU или nn.Tanh?
 

# Задание 1.2

Используя код из лекции, посмотрите, как будут вести функции активации nn.Tanh и nn.ReLU в случае инициализации весов при помощи He и Glorot.

Конкретно обратите внимание на распределение градиентов слоев и активаций.

Какие выводы можно сделать? Согласуются ли они с результатами из лекции?




## Задание 2
Реализуйте линейный слой с DropConnect. 

Проверьте, как он влияет на работу вашей нейросети. 

Сравните с работой nn.Dropout

In [ ]:
class DropConnect(nn.Module):
    def __init__(self, input_dim, output_dim, p=0.5):
        super(DropConnect, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim) # we have to keep linear level INSIDE Dropconnect
        self.p = p

    def forward(self, x):    
        size = self.linear.weight.size()
        if self.training:          
            keep = ... # some weight must be multiplied by 0
        else:
            keep = ... # all weight must be multiplied by p
            # no need in optimizations like w disccussed in the lecture
            # Yout layer just must work

        if self.cuda:
            keep = ... 
        
        output = F.linear(x, self.linear.weight * keep, self.linear.bias)
        return output

# Задание 3.1

Реализуйте Layer Normalization. Опять же, реализуем только для случая линейного слоя.

Вставьте слой в вашу нейросеть. Как он работает? 



In [ ]:
# полезные функции: .std(dim), .mean(dim)
class LayerNormalization(nn.Module):
    def __init__(self, in_d):
        super().__init__()
        self.gamma = nn.Parameter(torch.Tensor(in_d))
        self.beta = nn.Parameter(torch.Tensor(in_d))
        self.eps = 0.0000001
        self.reset_parameters()

    def reset_parameters(self):
        if self.gamma is not None:
            self.gamma.data.fill_(1)
        if self.beta is not None:
            self.beta.data.zero_()
  
    def forward(self, x):
        central_x =...
        # remove mean from each object batch-wise
        # use keepdim here

        var = ... + self.eps # to avoid overflow issues, calculate std through var
        std = ...
        normalized_x = ...

        out = ... * self.gamma
        out = ... + self.beta
        return out

# Задание 3.2 

Сравните с библиотечной реализацией nn.LayerNorm. Результаты должны быть похожи. Если у вас возникли затруднения со своей реализацией, это задание можно сдать без предыдущего


# Задание 4.1 

Выберите два адаптивных оптимизатора на выбор. Возьмите архитектуру, полученную в прошлом задании. 

Сравните качество обучения

## Задание 5.1 

Возьмите сверточную архитектуру из предыдущего занятия. Добавьте в нее BatchNorm/Dropout. Сравните качество с полученным на прошлом занятии и на этом при помощи обычной многослойной линейной сети

# Задание 5.2 

На лучшей из рассмотренных архитектур попробуйте добиться лучшего качества, применяя разные оптимизаторы и их настройки

# Задание 6 (необязательное)

Добавьте в вашу нейросеть Dropout. BatchNorm добавлять не надо, если он уже есть - уберите

Обучите ее. После этого, сделайте предсказание на тестовом датасете НЕ ДЕЛАЯ model.eval. Повторите процедуру 100 раз. 
Выведите 10 изображений, в классификации которых нейросеть больше всего неуверенна (на 100 запусков предсказывала им самые различные классы) 